In [1]:
import re
import sys
import json


import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
from nltk.lm        import Vocabulary
# from nltk.lm.models import MLE
from nltk.lm.models import KneserNeyInterpolated
from nltk.util      import ngrams

In [3]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
error_types = ['Unclear intention', 'Wrong information',
 'Ignore question', 'Topic transition error', 
 'Lack of information', 'Repetition', 
 'Contradiction', 'Self-contradiction',
  'Lack of common sense', 'Semantic error',
   'Grammatical error', 'Ignore proposal', 
   'Ignore offer', 'Lack of sociality', 
   'Uninterpretable', 'Ignore greeting', 
   'No-Err']

In [4]:
convs = read_conv(path, datalist)

In [121]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

error = "Grammatical error"
errors = ['Grammatical error', "Uninterpretable"]
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            if not ut.utt[-1] in ["？", "！", "。", "!"]:
                # sys_utt.append(ut.utt+"。")
                sys_utt.append(ut.utt)
            else:   
                sys_utt.append(ut.utt)
            if ut.is_error_included(errors):
                y.append(0)
            else:
                y.append(1)

In [118]:
print("len of y:{0}, error '{1}' counts:{2}".format(len(y), error, y.count(1)))

len of y:1349, error 'Grammatical error' counts:1345


In [99]:
modelM = DataManager("../models/utterance/")
# model_name = "KLM_nucc.pickle"
# model_name = "KLM_phrase.pickle"
n = 3
model_name = "KLM_phrase_n={0}.pickle".format(n)
# model_name = "KLM_phrase_n={0}_noun.pickle".format(n)
model_name = "KLM_phrase_nucc_n={0}.pickle".format(n)
model_name = "KLM_phrase_nucc_n={0}_noun.pickle".format(n)
lm = modelM.load_data(model_name)

success load : ../models/utterance/KLM_phrase_nucc_n=3_noun.pickle


In [100]:
import math
import pprint
def sentence2score(sentence, l, N):
    # filled = fill_SYMBOL( sentence2normalize_independent(sentence) )
    filled = fill_SYMBOL( sentence2normalize_noun(sentence) )
    filled_pos = fill_SYMBOL( sentence2pos(sentence) )
    # print(filled)
    # print(filled_pos)
    ngram_text = []
    ngram_pos = []
    for L, P in zip(filled,filled_pos):
        for i in range(len(L)-N+1):
            # print(L[i:i+N])
            ngram_text.append(L[i:i+N])
            ngram_pos.append(P[i:i+N])
    # pprint.pprint(ngram_text)
    all_score = 0
    function_score = 0
    # デフォルトで1
    function_num = 1
    for ngram, pgram in zip(ngram_text, ngram_pos):
        context = (ngram[:-1])
        context_pos = pgram[:-1]
        # print(context)
        # for word in lm.context_counts(lm.vocab.lookup(context)): # 文脈に続く単語一覧の取得
            
        score = lm.score(ngram[-1], context) + 1e-4
        log_score = math.log2(score)
        # print("context : {0}|{1} ->".format(context, ngram[-1:]), log_score)

        if "助動詞" in context_pos[1] or "助詞" in context_pos[1] or "助動詞" in context_pos[0] or "助詞" in context_pos[0]:
            # print("\tcontext : {0}| ->".format(context), log_score)
            function_score += log_score
            function_num += 1

        all_score += log_score
    # print(all_score/len(ngram_text))
    # return all_score/len(ngram_text)
    return function_score/function_num

In [101]:

sentence = "最近とても暑いですから。"
# sentence = "魚はおいしいんですか？？海は素晴らしいですね。"
# sentence = "魚はおいんですか？？海は素晴らしいですね。"
sentence2score(sentence, lm, N=n)

-3.6100002077774005

In [102]:
error_samples = [['ご存知ですいます。'],
 ['魚はおいんですか？？海は素晴らしいですね。'],
 ['もちろんですってのが元気ですかにいくないですよ。'],
 ['熱中症に気をつけか？？']]

for s in error_samples:
    print("{0}:{1}".format(s[0], sentence2score(s[0], lm, N=n)))

ご存知ですいます。:-5.243564695392782
魚はおいんですか？？海は素晴らしいですね。:-6.645624200019985
もちろんですってのが元気ですかにいくないですよ。:-6.351135445510097
熱中症に気をつけか？？:-6.889447202555353


- N=4 gram の全品詞，全単語
        
        ご存知ですいます:-8.056322000232084
        魚はおいんですか？？海は素晴らしいですね:-3.628491577687402
        もちろんですってのが元気ですかにいくないですよ:-8.338289348723574
        熱中症に気をつけか？？:-3.

- N=4 で助詞，助動詞の前後

        ご存知ですいます:-4.654287917265429
        魚はおいんですか？？海は素晴らしいですね:-3.56763363850194
        もちろんですってのが元気ですかにいくないですよ:-7.488756600321924
        熱中症に気をつけか？？:-3.9569609593484123

- N=4 gram の全品詞，全単語
    

In [103]:
# scoreを入れておこう
y_scores = []
for utt in sys_utt:
    y_scores.append(sentence2score(utt, lm, N=n))

In [124]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
import numpy as np

border = -6.8
epoch = 1
y_pred = np.zeros(len(y)) + 1
max_precision = 0

print(model_name)
for e in range(epoch):
    # y_pred = np.zeros(len(y))
    y_pred = np.zeros(len(y)) + 1
    for i, score in enumerate(y_scores):
        # border 未満をエラーでとする
        if score < border + 0.01*e :
            y_pred[i] = 0
            # print(sys_utt[i])
        # precision = metrics.precision_score(y, y_pred)

    # print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
    # print(border + 0.01*e)
    # print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
    # print()


print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
print('precision = ', precision_score(y_true=y, y_pred=y_pred))
print('recall = ', recall_score(y_true=y, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y, y_pred=y_pred))

KLM_phrase_nucc_n=3_noun.pickle
confusion matrix = 
 [[   4    0]
 [ 126 1219]]
accuracy =  0.9065974796145293
precision =  1.0
recall =  0.9063197026022305
f1 score =  0.9508580343213728


In [122]:
y.count(1)

1345

In [123]:
list(y_pred).count(1)

0

- corpus をゴガクルにした場合

        confusion matrix = 
        [[1300   45]
        [   2    2]]
        confusion matrix = 
        [[1206  139]
        [   2    2]]
        confusion matrix = 
        [[961 384]
        [  1   3]]
        confusion matrix = 
        [[363 982]
        [  0   4]]
        confusion matrix = 
        [[  86 1259]
        [   0    4]]

- 助詞，助動詞の直後のみ

        confusion matrix = 
        [[888 457]
        [  3   1]]
        confusion matrix = 
        [[1135  210]
        [   4    0]]
        confusion matrix = 
        [[1282   63]
        [   4    0]]
        confusion matrix = 
        [[1329   16]
        [   4    0]]
        confusion matrix = 
        [[1343    2]
        [   4    0]]

- 名詞を正規化

        confusion matrix = 
        [[1175  170]
        [   1    3]]
        confusion matrix = 
        [[1127  218]
        [   0    4]]
        confusion matrix = 
        [[1041  304]
        [   0    4]]
        confusion matrix = 
        [[921 424]
        [  0   4]]
        confusion matrix = 
        [[765 580]
        [  0   4]]
        confusion matrix = 
        [[568 777]
        [  0   4]]

- KLM_phrase_nucc_n=3_noun.pickle / return func

        confusion matrix = 
        [[1233  112]
        [   2    2]]
        confusion matrix = 
        [[1195  150]
        [   0    4]]
        confusion matrix = 

- border = -6.7 / KLM_phrase_nucc_n=3_noun.pickle

        confusion matrix = 
        [[   4    0]
        [ 126 1219]]
        accuracy =  0.9065974796145293
        precision =  1.0
        recall =  0.9063197026022305
        f1 score =  0.9508580343213728